<h1>IBM Applied Data Science Capstone Course by Coursera</h1>

<h2>Week 5 Final Report</h2>

<h3> Opening a New Restaurant in Toronto, Canada</h3>


<ol>
<li>Build a dataframe of neighborhoods in Toronto,canada by web scraping the data from Wikipedia page
<li>Get the geographical coordinates of the neighborhoods
<li>Obtain the venue data for the neighborhoods from Foursquare API
<li>Explore and cluster the neighborhoods
<li>Select the best cluster to open a new restaurant

<h5>1.Importing libraries

In [6]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



print("Libraries imported.")

Libraries imported.


<h5> 2.Getting the data from wikipedia

In [7]:
import requests # for http requests
import pandas as pd # for creating the dataframe
from bs4 import BeautifulSoup # for scrapping webdata

In [8]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')

In [9]:
table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]

Now let us clean the scrapped data to get a clean and untangled table .

table is then converted to a dataframe

In [10]:
df = pd.DataFrame(row)
df1 = df[0].str.split('\n', expand=True)
df2 = df1.rename(columns=df1.iloc[0])
df3 = df2.drop(df2.index[0])
df3.head()


,,Postcode,Borough,Neighbourhood,
1,,M1A,Not assigned,Not assigned,
2,,M2A,Not assigned,Not assigned,
3,,M3A,North York,Parkwoods,
4,,M4A,North York,Victoria Village,
5,,M5A,Downtown Toronto,Harbourfront,


In [11]:
##Inoring the cella that has NOT ASSIGNED IN BOROUGH COLUMN.
df4 = df3[df3.Borough != 'Not assigned']
df4.head()

,,Postcode,Borough,Neighbourhood,
3,,M3A,North York,Parkwoods,
4,,M4A,North York,Victoria Village,
5,,M5A,Downtown Toronto,Harbourfront,
6,,M5A,Downtown Toronto,Regent Park,
7,,M6A,North York,Lawrence Heights,


In [12]:
df5 = df4.groupby(['Postcode', 'Borough'], sort = False).agg(','.join)
df5.reset_index(inplace = True)
df5.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [13]:
df6=df5.replace('Not assigned','queens park')
df6.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,queens park


<h5>3.Getting the geographical cp-prdinates

In [14]:
url = "http://cocl.us/Geospatial_data"
df7 = pd.read_csv(url)
df7.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
##Changing the name of the second dataframe to eliminate error during merging
df7.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
df7.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
##Merging to get a single dataframe
df8 = pd.merge(df6, df7, on='Postcode')
df8.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,queens park,43.662301,-79.389494


<h5>4. Create a map of Toronto with neighborhoods superimposed on top

In [1]:
#installing geopy and folium
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [22]:
import geopy
from geopy.geocoders import Nominatim
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [23]:
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(df8['Latitude'],df8['Longitude'],df8['Postcode'],df8['Borough'],df8['Neighbourhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto



In [25]:
# save the map as HTML file
map_toronto.save('map_toronto.html')

<h5> 5. Use the Foursquare API to explore the neighborhoods</h5

In [28]:
# define Foursquare Credentials and Version
CLIENT_ID ='2BIIT5E5CH110VLOTA0FCJPMLBCJ2VIQVFT05TYEK1KZXBYJ' 
CLIENT_SECRET='5N00NXGTEMA4SPPQEYRKXFTNDH51J531QBTZROQYYZYJ2IFZ'
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2BIIT5E5CH110VLOTA0FCJPMLBCJ2VIQVFT05TYEK1KZXBYJ
CLIENT_SECRET:5N00NXGTEMA4SPPQEYRKXFTNDH51J531QBTZROQYYZYJ2IFZ


In [32]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df8['Latitude'],df8['Longitude'],df8['Neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [33]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(8561, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Parkwoods,43.753259,-79.329656,LA Fitness,43.747665,-79.347077,Gym / Fitness Center
3,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store
4,Parkwoods,43.753259,-79.329656,Graydon Hall Manor,43.763923,-79.342961,Event Space


In [35]:
##Let's check how many venues were returned for each neighorhood
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,100,100,100,100,100,100
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",100,100,100,100,100,100
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",48,48,48,48,48,48
"Alderwood,Long Branch",100,100,100,100,100,100
"Bathurst Manor,Downsview North,Wilson Heights",54,54,54,54,54,54
Bayview Village,49,49,49,49,49,49
"Bedford Park,Lawrence Manor East",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100


In [36]:
##Let's find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 326 uniques categories.


In [37]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Caribbean Restaurant', 'Golf Course', 'Gym / Fitness Center',
       'Liquor Store', 'Event Space', 'Middle Eastern Restaurant',
       'Supermarket', 'Park', 'Mediterranean Restaurant',
       'Japanese Restaurant', 'Café', 'Fast Food Restaurant',
       'Persian Restaurant', 'Seafood Restaurant', 'Coffee Shop',
       'Asian Restaurant', 'Ice Cream Shop', 'Steakhouse',
       'Chinese Restaurant', 'Burger Joint', 'Hotel',
       'American Restaurant', 'Pizza Place', 'Pool Hall',
       'Discount Store', 'Gym', 'Pharmacy', 'Italian Restaurant', 'Bank',
       'Salad Place', 'Paper / Office Supplies Store', 'Hakka Restaurant',
       'Grocery Store', 'Beer Store', 'Fried Chicken Joint',
       'Sandwich Place', 'Fish & Chips Shop', 'Thai Restaurant',
       'Automotive Shop', 'Bar', 'Breakfast Spot',
       'Furniture / Home Store', 'Greek Restaurant', 'Road', 'Bus Stop',
       'Sushi Restaurant', 'Skating Rink', 'Tennis Court', 'Wings Joint',
       'Restaurant'], dtype=objec

In [38]:
# check if the results contain "Restaurant"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

</h4> 6.Analyze Each Neighborhood </h4>

In [39]:
# one hot encoding
tr_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tr_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tr_onehot.columns[-1]] + list(tr_onehot.columns[:-1])
tr_onehot = tr_onehot[fixed_columns]

print(tr_onehot.shape)
tr_onehot.head()

(8561, 327)


,Neighborhoods,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri 

In [40]:
tr_grouped = tr_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(tr_grouped.shape)
tr_grouped

(103, 327)


,Neighborhoods,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri 

In [41]:
len(tr_grouped[tr_grouped["Shopping Mall"] > 0])

46

<h5>
Create a new DataFrame for Shopping Mall data only</h5>

In [44]:
tr_Restaurant= tr_grouped[["Neighborhoods","Restaurant"]]
tr_Restaurant.head()

,Neighborhoods,Restaurant
0,"Adelaide,King,Richmond",0.03
1,Agincourt,0.04
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.00
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.00
4,"Alderwood,Long Branch",0.02


<h5> 7.Clustering Neughbourhoods</h5>

Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [47]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 3

tr_clustering = tr_Restaurant.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tr_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 2, 2, 0, 1, 2, 1, 1, 0], dtype=int32)

In [49]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
tr_merged = tr_Restaurant.copy()

# add clustering labels
tr_merged["Cluster Labels"] = kmeans.labels_

In [58]:
tr_merged.rename(columns={"Neighborhoods": "Neighbourhood"}, inplace=True)
tr_merged.head()

,Neighborhood,Restaurant,Cluster Labels
0,"Adelaide,King,Richmond",0.03,1
1,Agincourt,0.04,1
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.00,2
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.00,2
4,"Alderwood,Long Branch",0.02,0


In [60]:
df8.rename(columns={"Neighbourhood": "Neighborhood"}, inplace=True)
df8.head()
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tr_merged = tr_merged.join(df8.set_index("Neighborhood"), on="Neighborhood")
print(tr_merged.shape)
tr_merged.head() # check the last columns!

(103, 7)


,Neighborhood,Restaurant,Cluster Labels,Postcode,Borough,Latitude,Longitude
0,"Adelaide,King,Richmond",0.03,1,M5H,Downtown Toronto,43.650571,-79.384568
1,Agincourt,0.04,1,M1S,Scarborough,43.794200,-79.262029
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.00,2,M1V,Scarborough,43.815252,-79.284577
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.00,2,M9V,Etobicoke,43.739416,-79.588437
4,"Alderwood,Long Branch",0.02,0,M8W,Etobicoke,43.602414,-79.543484


In [62]:
# sort the results by Cluster Labels
print(tr_merged.shape)
tr_merged.sort_values(["Cluster Labels"], inplace=True)
tr_merged.head()

(103, 7)


,Neighborhood,Restaurant,Cluster Labels,Postcode,Borough,Latitude,Longitude
102,queens park,0.01,0,M7A,Queen's Park,43.662301,-79.389494
27,"Commerce Court,Victoria Hotel",0.02,0,M5L,Downtown Toronto,43.648198,-79.379817
28,Davisville,0.02,0,M4S,Central Toronto,43.704324,-79.388790
58,Humewood-Cedarvale,0.02,0,M6C,York,43.693781,-79.428191
30,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0.02,0,M4V,Central Toronto,43.686412,-79.400049


In [65]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tr_merged['Latitude'],tr_merged['Longitude'],tr_merged['Neighborhood'],tr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [66]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

<h5>8. Examine Clusters</h5>

<h5>cluster 0</h5>

In [67]:
tr_merged.loc[tr_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Postcode,Borough,Latitude,Longitude
102,queens park,0.010000,0,M7A,Queen's Park,43.662301,-79.389494
27,"Commerce Court,Victoria Hotel",0.020000,0,M5L,Downtown Toronto,43.648198,-79.379817
28,Davisville,0.020000,0,M4S,Central Toronto,43.704324,-79.388790
58,Humewood-Cedarvale,0.020000,0,M6C,York,43.693781,-79.428191
30,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0.020000,0,M4V,Central Toronto,43.686412,-79.400049
31,"Del Ray,Keelesdale,Mount Dennis,Silverthorn",0.014085,0,M6M,York,43.691116,-79.476013
32,"Design Exchange,Toronto Dominion Centre",0.020000,0,M5K,Downtown Toronto,43.647177,-79.381576
79,"Ryerson,Garden District",0.020000,0,M5B,Downtown Toronto,43.657162,-79.378937
34,"Dorset Park,Scarborough Town Centre,Wexford He...",0.010638,0,M1P,Scarborough,43.757410,-79.273304
35,"Dovercourt Village,Dufferin",0.020000,0,M6H,West Toronto,43.669005,-79.442259


<h5>cluster 1</h5>

In [70]:
tr_merged.loc[tr_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Postcode,Borough,Latitude,Longitude
65,Leaside,0.036145,1,M4G,East York,43.709060,-79.363452
63,"Lawrence Heights,Lawrence Manor",0.030000,1,M6A,North York,43.718518,-79.464763
66,"Little Portugal,Trinity",0.040000,1,M6J,West Toronto,43.647927,-79.419750
67,"Maryvale,Wexford",0.030303,1,M1R,Scarborough,43.750072,-79.295849
73,"Parkdale,Roncesvalles",0.050000,1,M6R,West Toronto,43.648960,-79.456325
91,Thorncliffe Park,0.040000,1,M4H,East York,43.705369,-79.349372
71,Northwest,0.027027,1,M9W,Etobicoke,43.706748,-79.594054
72,"Northwood Park,York University",0.051282,1,M3J,North York,43.767980,-79.487262
89,"The Junction North,Runnymede",0.027778,1,M6N,York,43.673185,-79.487262
75,Rosedale,0.030000,1,M4W,Downtown Toronto,43.679563,-79.377529


<h5>cluster 2</h5>

In [69]:
tr_merged.loc[tr_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Postcode,Borough,Latitude,Longitude
40,"East Birchmount Park,Ionview,Kennedy Park",0.0,2,M1K,Scarborough,43.727929,-79.262029
100,Woodbine Heights,0.0,2,M4C,East York,43.695344,-79.318389
62,L'Amoreaux West,0.0,2,M1W,Scarborough,43.799525,-79.318389
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,2,M1V,Scarborough,43.815252,-79.284577
97,Willowdale West,0.0,2,M2R,North York,43.782736,-79.442259
96,Willowdale South,0.0,2,M2N,North York,43.770120,-79.408493
95,Weston,0.0,2,M9N,York,43.706876,-79.518188
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,2,M9V,Etobicoke,43.739416,-79.588437
93,Victoria Village,0.0,2,M4A,North York,43.725882,-79.315572
92,Upper Rouge,0.0,2,M1X,Scarborough,43.836125,-79.205636


observations:
The above dataframe shows that most of the restaurants are in the central loction of Toronto city,with the highest number of cluster 2 and moderate number in cluster 0. On the other hand, cluster 1 has very low number to totally no Restaurant in the neighborhoods.This represents a great opportunity and high potential areas to open new Restaurant as there is very little to no competition from existing restaurant.Meanwhile, restaurant in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of restaurant.Property developers with unique selling propositions to stand out from the competition can also open new restaurant in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of Restaurant and suffering from intense competition

